# SVM From Scratch

<br>

<img src="images/svm_concept_2.jpeg" width='700' />

<br>
<br>

<img src="images/svm_model.png" width='400' />
<center>SVM Model Expressed Mathematically</center>

In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle

In [18]:
# >> FEATURE SELECTION << #
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped


def remove_less_significant_features(X, Y):
    sl = 0.05
    regression_ols = None
    columns_dropped = np.array([])
    for itr in range(0, len(X.columns)):
        regression_ols = sm.OLS(Y, X).fit()
        max_col = regression_ols.pvalues.idxmax()
        max_val = regression_ols.pvalues.max()
        if max_val > sl:
            X.drop(max_col, axis='columns', inplace=True)
            columns_dropped = np.append(columns_dropped, [max_col])
        else:
            break
    regression_ols.summary()
    return columns_dropped


##############################

## Cost Function (Objective Function)

Our objective is to find a hyperplane that separates +ve and -ve examples with the largest margin while keeping the misclassification as low as possible.

We will minimize the cost/objective function shown below:

<br>

<img src="images/cost_function.png" width='600' />

**<center>In the training phase, Larger C results in the narrow margin (for infinitely large C the SVM becomes hard margin) and smaller C results in the wider margin.</center>**

<br>
<br>

Another version of a cost function
<img src="images/cost_function_2.png" width='600' />



**<center>Larger λ gives a wider margin and smaller λ results in the narrow margin (for infinitely small λ the SVM becomes hard margin).</center>**
In this cost function, λ is essentially equal to 1/C and has the opposite effect.

## The Gradient of the Cost Function

<img src="images/gradiant_of_CF.png" width='700' />

In [19]:
# >> MODEL TRAINING << #
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost


# I haven't tested it but this same function should work for
# vanilla and mini-batch gradient descent as well
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw


def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights


########################

## Reading the Dataset

In [20]:
 data = pd.read_csv('data.csv')

<br>

Drop last column (extra column added by pd) and unnecessary first column (id)

In [21]:
data.drop(data.columns[[-1, 0]], axis=1, inplace=True)

<br>

SVM only accepts numerical values. Therefore, we will transform the categories M and B into values 1 and -1 respectively.

In [22]:
diag_map = {'M': 1.0, 'B': -1.0}
data['diagnosis'] = data['diagnosis'].map(diag_map)

## Feature Engineering

Feature engineering techniques have two characteristics in common:

- Preparing the data which is compatible with the model
- Improving the performance of the machine learning algorithm

**Normalization** is one of the many feature engineering technique. Normalization is the process of converting a range of values, into a standard range of values, typically in the interval [−1, 1] or [0, 1]. It’s not a strict requirement but it improves the speed of learning (e.g. faster convergence in gradient descent) and prevents numerical overflow.

In [23]:
Y = data.loc[:, 'diagnosis']
X = data.iloc[:, 1:]

In [24]:
# filter features
remove_correlated_features(X)
remove_less_significant_features(X, Y)

array(['smoothness_mean', 'compactness_worst', 'compactness_mean',
       'radius_mean', 'texture_se', 'symmetry_se', 'smoothness_se',
       'concavity_worst'], dtype='<U32')

In [25]:
# normalize the features using MinMaxScalar

X_normalized = MinMaxScaler().fit_transform(X.values)
X = pd.DataFrame(X_normalized)

In [26]:
# insert 1 in every row for intercept b
X.insert(loc=len(X.columns), column='intercept', value=1)

## Splitting the Dataset

In [27]:
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)

In [29]:
# set hyper-parameters and call init
regularization_strength = 10000
learning_rate = 0.000001

In [30]:
# train the model
print("training started...")
W = sgd(X_train.to_numpy(), y_train.to_numpy())
print("training finished.")
print("weights are: {}".format(W))

training started...
Epoch is: 1 and Cost is: 7241.912513637917
Epoch is: 2 and Cost is: 6543.973619707361
Epoch is: 4 and Cost is: 5435.448266033489
Epoch is: 8 and Cost is: 3836.385742838332
Epoch is: 16 and Cost is: 2637.9997945349855
Epoch is: 32 and Cost is: 1966.732146494961
Epoch is: 64 and Cost is: 1584.9800341912085
Epoch is: 128 and Cost is: 1323.5706207185253
Epoch is: 256 and Cost is: 1160.324596925277
Epoch is: 512 and Cost is: 1077.2306308348145
Epoch is: 1024 and Cost is: 1047.594733753037
Epoch is: 2048 and Cost is: 1054.5711198462566
training finished.
weights are: [ 3.53866577 11.05131892 -2.33022667 -7.92293421 10.14360543 -1.31074021
 -6.45350098  2.23130273 -3.89423401  3.21090318  4.94062452  4.80083756
 -4.7922664 ]


In [32]:
# testing the model
print("testing the model...")
y_train_predicted = np.array([])
for i in range(X_train.shape[0]):
    yp = np.sign(np.dot(X_train.to_numpy()[i], W))
    y_train_predicted = np.append(y_train_predicted, yp)

    y_test_predicted = np.array([])
for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(X_test.to_numpy()[i], W))
    y_test_predicted = np.append(y_test_predicted, yp)

print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
print("precision on test dataset: {}".format(recall_score(y_test, y_test_predicted)))

testing the model...
accuracy on test dataset: 0.9736842105263158
recall on test dataset: 0.9302325581395349
precision on test dataset: 0.9302325581395349
